In [7]:
import pandas as pd
import math

In [3]:
def null_pct(df):
    return dict(zip(df.columns, [round(100*len(df.loc[df[c].isnull()])/len(df),2) for c in df.columns]))

In [4]:
pen_path_2021 = "./PEN America's Index of School Book Bans (July 1, 2021 - June 30, 2022).xlsx"
pen_path_2022 = "./PEN America's Index of School Book Bans (July 1, 2022 - June 30, 2023).xlsx"
pen_21_df = pd.read_excel(pen_path_2021, sheet_name='Sorted by Author & Title', skiprows=2)
pen_22_df = pd.read_excel(pen_path_2022, sheet_name='Sorted by Author & Title', skiprows=2)
pen_df = pd.concat((pen_21_df, pen_22_df), axis=0, ignore_index=True)\
    .drop(['Secondary Author(s)', 'Illustrator(s)', 'Translator(s)', 'Series Name', 'Origin of Challenge'], axis=1)
print(null_pct(pen_df))

{'Author': 0.02, 'Title': 0.02, 'Type of Ban': 57.04, 'State': 0.0, 'District': 0.0, 'Date of Challenge/Removal': 0.0, 'Ban Status': 42.96}


In [5]:
pen_df = pen_df.rename(columns={'Date of Challenge/Removal': 'DateBan'}).drop_duplicates()
pen_df['Country'] = 'USA'
date_dict = {"AY 2022-2023":"2022-09-01", "Fall 2022":"2022-09-01", "Spring 2023":"2023-01-01"}
pen_df['DateBan'] = pen_df['DateBan'].replace(date_dict)
pen_df['DateBan'] = pd.to_datetime(pen_df['DateBan']).dt.to_period('D')
print(pen_df.shape)
pen_df

(5894, 8)


,Author,Title,Type of Ban,State,District,DateBan,Ban Status,Country
0,"Àbíké-Íyímídé, Faridah",Ace of Spades,Banned in Libraries and Classrooms,Florida,Indian River County School District,2021-11-01,NaN,USA
1,"Acevedo, Elizabeth",Clap When You Land,Banned in Classrooms,Pennsylvania,Central York School District,2021-08-31,NaN,USA
2,"Acevedo, Elizabeth",The Poet X,Banned in Libraries,Florida,Indian River County School District,2021-11-01,NaN,USA
3,"Acevedo, Elizabeth",The Poet X,Banned in Libraries and Classrooms,New York,Marlboro Central School District,2022-02-01,NaN,USA
4,"Acevedo, Elizabeth",The Poet X,Banned Pending Investigation,Texas,Fredericksburg Independent School District,2022-03-01,NaN,USA
...,...,...,...,...,...,...,...,...
5889,"Zoboi, Ibi",My Life as an Ice Cream Sandwich,NaN,Florida,Clay County School District,2022-09-01,Banned Pending Investigation,USA
5890,"Zoboi, Ibi",Punching the Air,NaN,Florida,Clay County School District,2022-09-01,Banned Pending Investigation,USA
5891,Zonderkidz.com,"Poisonous, Smelly, and Amazing Plants",NaN,Florida,Brevard Public Schools,2023-06-01,Banned from Libraries and Classrooms,USA
5892,"Zuffi, Stefano",Bosch: The Garden of Earthly Delights,NaN,Missouri,Wentzville School District,2022-09-01,Banned Pending Investigation,USA


In [6]:
#pen_df.groupby('State').size().sort_values(ascending=False).plot.bar()
pen_df = pen_df.loc[~pen_df['DateBan'].apply(lambda x: isinstance(x, str)),:]
print(len(pen_df))

5894


In [11]:
def make_desc(type_of_ban, ban_status):
    desc = ""
    if type(type_of_ban)==str or not math.isnan(type_of_ban):
        desc+=type_of_ban
    if type(ban_status)==str or not math.isnan(ban_status):
        desc+= ban_status
    
    return desc if desc else None
pen_df["Description"] = pen_df.apply(lambda x: make_desc(x['Type of Ban'], x['Ban Status']), axis=1)

In [14]:
pen_df = pen_df.drop(columns=["Type of Ban", "Ban Status"])
pen_df

,Author,Title,State,District,DateBan,Country,Description
0,"Àbíké-Íyímídé, Faridah",Ace of Spades,Florida,Indian River County School District,2021-11-01,USA,Banned in Libraries and Classrooms
1,"Acevedo, Elizabeth",Clap When You Land,Pennsylvania,Central York School District,2021-08-31,USA,Banned in Classrooms
2,"Acevedo, Elizabeth",The Poet X,Florida,Indian River County School District,2021-11-01,USA,Banned in Libraries
3,"Acevedo, Elizabeth",The Poet X,New York,Marlboro Central School District,2022-02-01,USA,Banned in Libraries and Classrooms
4,"Acevedo, Elizabeth",The Poet X,Texas,Fredericksburg Independent School District,2022-03-01,USA,Banned Pending Investigation
...,...,...,...,...,...,...,...
5889,"Zoboi, Ibi",My Life as an Ice Cream Sandwich,Florida,Clay County School District,2022-09-01,USA,Banned Pending Investigation
5890,"Zoboi, Ibi",Punching the Air,Florida,Clay County School District,2022-09-01,USA,Banned Pending Investigation
5891,Zonderkidz.com,"Poisonous, Smelly, and Amazing Plants",Florida,Brevard Public Schools,2023-06-01,USA,Banned from Libraries and Classrooms
5892,"Zuffi, Stefano",Bosch: The Garden of Earthly Delights,Missouri,Wentzville School District,2022-09-01,USA,Banned Pending Investigation


In [15]:
pen_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5894 entries, 0 to 5893
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype    
---  ------       --------------  -----    
 0   Author       5893 non-null   object   
 1   Title        5893 non-null   object   
 2   State        5894 non-null   object   
 3   District     5894 non-null   object   
 4   DateBan      5894 non-null   period[D]
 5   Country      5894 non-null   object   
 6   Description  5894 non-null   object   
dtypes: object(6), period[D](1)
memory usage: 497.4+ KB


In [16]:
pen_df.to_csv("PEN.csv")